to patch a text file with a diff file, use the following command:
<br/>
<br/>
```    patch 0.txt 1.diff ```

In [160]:
import json
import mwclient
import subprocess
from nltk import tokenize

In [164]:
# collect data
site = mwclient.Site('en.wikipedia.org')
page = site.pages['Wikipedia']

for i, (info, content) in enumerate(zip(page.revisions(), page.revisions(prop='content'))):
    # First 100 revisions
    if i > 100:
        break
    with open(f'data/{i}.json', "w") as f:
        f.write(json.dumps({ 'info': info, 'content': content}, indent=4))
    string = json.dumps(json.loads(open(f'data/{i}.json').read())['content']['*'], indent=4)
    with open(f'data/{i}.txt', "w") as f:
        f.write('\n'.join(tokenize.sent_tokenize(string)))

In [165]:
# generate diffs
old = ''.join([line + "\n" for line in open(f'data/{0}.txt').readlines()])

for i in range(100):
    new = ''.join([line + "\n" for line in open(f'data/{i+1}.txt').readlines()])
    diff_process = subprocess.run(['bash', '-c', f'diff <( printf "%s\n" {old} ) <( printf "%s\n" {new} )'], stdout=subprocess.PIPE)
    diff = diff_process.stdout.decode('utf-8')
    open(f'data/{i}.diff', "w").write(diff)
    old = new